In [1]:
import findspark
findspark.init("/Users/rahmanidriss/bigdata/tools/spark")
#findspark.init()
#import pyspark
#import SparkSession
from pyspark.sql import Row, SparkSession


In [2]:

spark = (
    SparkSession.builder
    #.master("spark://172.30.0.3:7077")
    #.config("spark.cores.max", 12)
    #.config("spark.executor.cores", 2)
    #.config("spark.executor.memory", "2g")
    .config("spark.sql.adaptive.enabled",'false')
    #.config('spark.sql.adaptive.coalescePartitions.initialPartitionNum', 24)
    #.config('spark.sql.adaptive.coalescePartitions.parallelismFirst', 'false')
    #.config('spark.sql.files.minPartitionNum', 1)
    #.config('spark.sql.files.maxPartitionBytes', '500mb')
    .appName('repartition_strategies')
    .getOrCreate()
)
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 13:44:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/04 13:44:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/04 13:44:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/04 13:44:31 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [31]:
spark.stop()

In [32]:
sc = spark.sparkContext
print(sc.defaultParallelism)

AttributeError: 'NoneType' object has no attribute 'sc'

In [18]:
spark.conf.get("spark.sql.shuffle.partitions")
#spark.conf.get("spark.executor.cores")

'200'

In [4]:
from pyspark.sql.types import  StructType,StructField,StringType,DoubleType, IntegerType,TimestampType, _parse_datatype_string, _parse_datatype_json_string
import pyspark.sql.functions as F



In [5]:
#_schema=StructType([StructField(c, IntegerType(), True) for c in df.columns])

_schema=StructType([
    StructField('MONTH', IntegerType(), True),
    StructField('DAY', IntegerType(), True),
    StructField('WEEKDAY', IntegerType(), True),
    StructField('AIRLINE', StringType(), True),
    StructField('ORG_AIR', StringType(), True),
    StructField('DEST_AIR', StringType(), True),
    StructField('SCHED_DEP', TimestampType(), True),
    StructField('DEP_DELAY', DoubleType(), True),
    StructField('AIR_TIME', DoubleType(), True),
    StructField('DIST', IntegerType(), True), 
    StructField('SCHED_ARR', IntegerType(), True),
    StructField('ARR_DELAY', DoubleType(), True),
    StructField('DIVERTED', IntegerType(), True),
    StructField('CANCELLED', IntegerType(), True)
    ])

                   

In [6]:
df=(
    spark.read
    .format("csv")
    .options(header="true",inferSschema="false",sep=",")
    .schema(_schema)
    .load("file:///Users/rahmanidriss/Desktop/dataScience/spark_hadoop_cluster/data/flights.csv")
)

In [7]:
df.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- WEEKDAY: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- ORG_AIR: string (nullable = true)
 |-- DEST_AIR: string (nullable = true)
 |-- SCHED_DEP: timestamp (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DIST: integer (nullable = true)
 |-- SCHED_ARR: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CANCELLED: integer (nullable = true)



In [8]:
df.show(3)

+-----+---+-------+-------+-------+--------+-------------------+---------+--------+----+---------+---------+--------+---------+
|MONTH|DAY|WEEKDAY|AIRLINE|ORG_AIR|DEST_AIR|          SCHED_DEP|DEP_DELAY|AIR_TIME|DIST|SCHED_ARR|ARR_DELAY|DIVERTED|CANCELLED|
+-----+---+-------+-------+-------+--------+-------------------+---------+--------+----+---------+---------+--------+---------+
|    1|  1|      4|     WN|    LAX|     SLC|1625-01-01 00:00:00|     58.0|    94.0| 590|     1905|     65.0|       0|        0|
|    1|  1|      4|     UA|    DEN|     IAD|               NULL|      7.0|   154.0|1452|     1333|    -13.0|       0|        0|
|    1|  1|      4|     MQ|    DFW|     VPS|1305-01-01 00:00:00|     36.0|    85.0| 641|     1453|     35.0|       0|        0|
+-----+---+-------+-------+-------+--------+-------------------+---------+--------+----+---------+---------+--------+---------+
only showing top 3 rows



In [67]:
# (df.agg(*(F.count_distinct(F.col(c)).alias(c) for c in df.columns))
#  .show()
# )

(df.agg(F.count_distinct("DEST_AIR"))
 .show()
)

+------------------------+
|count(DISTINCT DEST_AIR)|
+------------------------+
|                     271|
+------------------------+



In [57]:
spark.createDataFrame([([1, 2, 3],),([1],),([],)], ['data']).show()

+---------+
|     data|
+---------+
|[1, 2, 3]|
|      [1]|
|       []|
+---------+



In [9]:


empty_df = spark.createDataFrame([(1,0)], ["id","value"])
df_new = empty_df.withColumn('value', F.explode(F.array_repeat(F.col("value"), 1000000)))
df_new=df_new.drop(("id"))


In [7]:
df_new.printSchema()

root
 |-- value: long (nullable = true)



In [10]:
df_add=spark.createDataFrame([(0,5000),(1,10000),(2,100000)],["id","value"])
df_add=df_add.drop("id")

In [11]:
df_final=df_new.union(df_add)


In [20]:
round_robin_partition=df_final.repartition(4)
round_robin_partition = (round_robin_partition
# here I will insert the partition logic
    .withColumn("partition_id", F.spark_partition_id()) # applying SQL built-in function to determine actual partition
    .groupBy(F.col("partition_id"))
    .agg(
      F.count(F.col("value")).alias("count"),
      F.min(F.col("value")).alias("min_value"),
      F.max(F.col("value")).alias("max_value")
      )
    .orderBy(F.asc(F.col("partition_id")))
)
round_robin_partition.explain()
round_robin_partition.rdd.getNumPartitions()

== Physical Plan ==
*(5) Sort [partition_id#195 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(partition_id#195 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=326]
   +- *(4) HashAggregate(keys=[partition_id#195], functions=[count(value#105L), min(value#105L), max(value#105L)])
      +- Exchange hashpartitioning(partition_id#195, 200), ENSURE_REQUIREMENTS, [plan_id=322]
         +- *(3) HashAggregate(keys=[partition_id#195], functions=[partial_count(value#105L), partial_min(value#105L), partial_max(value#105L)])
            +- *(3) Project [value#105L, SPARK_PARTITION_ID() AS partition_id#195]
               +- Exchange RoundRobinPartitioning(4), REPARTITION_BY_NUM, [plan_id=317]
                  +- Union
                     :- *(1) Generate explode(array_repeat(value#101L, 1000000)), false, [value#105L]
                     :  +- *(1) Project [value#101L]
                     :     +- *(1) Scan ExistingRDD[id#100L,value#101L]
                     +- *(2) Project [valu

4

In [24]:
hash_partition=df_final.repartition(8, F.col("value"))
hash_partition = (hash_partition
# here I will insert the partition logic
    .withColumn("partition_id", F.spark_partition_id()) # applying SQL built-in function to determine actual partition
    .groupBy(F.col("partition_id"))
    .agg(
      F.count(F.col("value")).alias("count"),
      F.min(F.col("value")).alias("min_value"),
      F.max(F.col("value")).alias("max_value")
      )
    .orderBy(F.asc(F.col("partition_id")))
)
hash_partition.show()
print(hash_partition.rdd.getNumPartitions())
print(hash_partition.count())
#hash_partition.rdd.toDebugString()

4
4


b'(4) MapPartitionsRDD[193] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[192] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  SQLExecutionRDD[191] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  MapPartitionsRDD[190] at javaToPython at NativeMethodAccessorImpl.java:0 []\n |  ShuffledRowRDD[189] at javaToPython at NativeMethodAccessorImpl.java:0 []\n +-(200) MapPartitionsRDD[188] at javaToPython at NativeMethodAccessorImpl.java:0 []\n     |   MapPartitionsRDD[184] at javaToPython at NativeMethodAccessorImpl.java:0 []\n     |   ShuffledRowRDD[183] at javaToPython at NativeMethodAccessorImpl.java:0 []\n     +-(8) MapPartitionsRDD[182] at javaToPython at NativeMethodAccessorImpl.java:0 []\n        |  MapPartitionsRDD[181] at javaToPython at NativeMethodAccessorImpl.java:0 []\n        |  ShuffledRowRDD[180] at javaToPython at NativeMethodAccessorImpl.java:0 []\n        +-(24) MapPartitionsRDD[179] at javaToPython at NativeMethodAccessorIm

In [30]:
"""
Due to performance reasons this method uses sampling to estimate 
the ranges. Hence, the output may not be consistent,
since sampling can return different values. 
The sample size can be controlled by the 
config spark.sql.execution.rangeExchange.sampleSizePerPartition.

"""

range_partition=df_final.repartitionByRange(8, F.col("value"))
range_partition = (range_partition
# here I will insert the partition logic
    .withColumn("partition_id", F.spark_partition_id()) # applying SQL built-in function to determine actual partition
    .groupBy(F.col("partition_id"))
    .agg(
      F.count(F.lit("value")).alias("count"),
      #F.count(F.lit(1)).alias("count"),
      F.min(F.col("value")).alias("min_value"),
      F.max(F.col("value")).alias("max_value")
      )
    .orderBy(F.asc(F.col("partition_id")))
)
range_partition.show()
print(range_partition.rdd.getNumPartitions())
print(range_partition.count())

#df_final.groupBy("value").agg(F.count(F.lit(1))).show()

+------------+-------+---------+---------+
|partition_id|  count|min_value|max_value|
+------------+-------+---------+---------+
|           0|1000000|        0|        0|
|           1|      1|     5000|     5000|
|           2|      1|    10000|    10000|
|           3|      1|   100000|   100000|
+------------+-------+---------+---------+

4
4
+------+--------+
| value|count(1)|
+------+--------+
|     0| 1000000|
| 10000|       1|
|100000|       1|
|  5000|       1|
+------+--------+



In [80]:
(F.rand() * spark.conf.get("spark.sql.shuffle.partitions")).cast("int")


Column<'CAST((rand(7973342854009252421) * 200) AS INT)'>

In [83]:
df_left = (df_final.withColumn("salt", (F.rand() * spark.conf.get("spark.sql.shuffle.partitions")).cast("int"))
.withColumn("partition_id",F.spark_partition_id()).show()
)

+-----+----+------------+
|value|salt|partition_id|
+-----+----+------------+
|    0|  23|          11|
|    0| 130|          11|
|    0|  14|          11|
|    0| 132|          11|
|    0|  91|          11|
|    0|  35|          11|
|    0| 188|          11|
|    0| 129|          11|
|    0| 183|          11|
|    0| 138|          11|
|    0|  25|          11|
|    0| 196|          11|
|    0|  68|          11|
|    0|  16|          11|
|    0|  77|          11|
|    0| 145|          11|
|    0|  47|          11|
|    0|  90|          11|
|    0|  38|          11|
|    0| 157|          11|
+-----+----+------------+
only showing top 20 rows

